In [51]:
from brian2 import *
import numpy as np
import warnings

#warnings.filterwarnings("ignore", category=UserWarning, module='brian2.codegen.generators.base')

start_scope()

defaultclock.dt = 0.0001*ms  

# Custom timing function
@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, result=1, sum=1, spikes_received=1)
def spike_timing(w, global_clock, layer, sum, spikes_received): 
    #print(global_clock)
    x = global_clock % 1
    if w >= 0:
        return (x ** (1 - w)) 
    else:
        return (1 - (1 - x) ** (1 + w)) 
    
@implementation('numpy', discard_units=True)
@check_units(layer=1, result=1, sum=1, spikes_received=1)
def math1(layer, sum, spikes_received): 
    return (sum/spikes_received )+ layer


In [52]:
# Urd, Verdande, Skuld 

# did not apply the specficyed weifhts to the first layer yet will maybe do after
def run_Urd(inputs, weights_1, weights_2, weights_3):
    '''4-10-3 SNN'''
    # will add check of weights # so it all works
    n_input = 4 
    n_hidden = 10
    n_output = 3
    n_total = n_input + n_hidden + n_output

    neurons = NeuronGroup(n_total, '''
        v : 1
        sum : 1
        spikes_received : 1
        scheduled_time : second
        global_clock : 1
    ''', threshold='v > 1', reset='v = 0', method='exact')
    neurons.v = 0
    neurons.scheduled_time = 1e9 * second
    neurons.global_clock = 0.0
    neurons.sum = 0.0
    neurons.spikes_received = 0.0


    indicess = [i for i in range(n_input)]
    stim = SpikeGeneratorGroup(n_input, indices=indicess, times=(inputs*ms))

    syn_input = Synapses(stim, neurons[0:n_input], '''
        w : 1
        layer : 1
    ''', on_pre='''
        spikes_received += 1
        sum += spike_timing(w, global_clock, layer, spikes_received, sum)
        scheduled_time = ((sum/spikes_received) + layer) * ms 
    ''')
    syn_input.connect(j='i')
    syn_input.w = weights_1
    syn_input.layer = 0

    syn_hidden = Synapses(neurons[0:n_input], neurons[n_input:n_input+n_hidden], '''
        w : 1
        layer : 1
    ''', on_pre='''
        spikes_received += 1
        sum += spike_timing(w, global_clock, layer, spikes_received, sum)
        scheduled_time = ((sum/spikes_received) + layer) * ms 
    ''')
    for inp in range(n_input):
        for hid in range(n_hidden):
            syn_hidden.connect(i=inp, j=hid)

    syn_hidden.w = weights_2
    syn_hidden.layer = 1


    syn_output = Synapses(
        neurons[n_input:n_input+n_hidden], 
        neurons[n_input+n_hidden:n_total], 
        '''
        w : 1
        layer : 1
        ''',
        on_pre='''
        spikes_received += 1
        sum += spike_timing(w, global_clock, layer, spikes_received, sum)
        scheduled_time = ((sum/spikes_received) + layer) * ms 
        '''
    )

    for hid in range(n_hidden):
        for out in range(n_output):
            syn_output.connect(i=hid, j=out)

    # Set weights in correct order
    syn_output.w[:] = weights_3
    syn_output.layer = 2

    #print(syn_output.i[:], syn_output.j[:])
    #weights_into_output_1 = weights_3[1::3]



    neurons.run_regularly('''
        v = int(abs(t - scheduled_time) < 0.0005*ms) * 1.2
        global_clock += 0.001
    ''', dt=0.001*ms)


    spikemon = SpikeMonitor(neurons)
    
    # neurons.v = 0
    # neurons.scheduled_time = 1e9 * second
    # neurons.global_clock = 0.0
    # neurons.sum = 0.0
    # neurons.spikes_received = 0.0

    run(5*ms)

    result = []

    for i in range(n_total):
        times = spikemon.spike_trains()[i]
        if len(times) > 0:
            result.append(round(times[0]/ms, 3))
        else:
            result.append(None)  # or some other placeholder like float('nan')
            
    return result


def calc_cost(outputs, desired_outputs):
    return 0.5 * ((outputs - desired_outputs) ** 2)


# Layer 1 weights: input (4 features) → hidden layer
w_1 = np.random.uniform(0.05, 0.95, size=4).tolist()   # small positive values

# Layer 2 weights: hidden layer → next layer
w_2 = np.random.uniform(-0.95, 0.95, size=40).tolist()  # symmetric, allows +/- contribution

# Layer 3 weights: next layer → output layer
w_3 = np.random.uniform(-0.95, 0.95, size=30).tolist()



def update_weights_input_neuron(hidden_activations, weights, actual_output, desired_output, learning_rate=0.01):

    """
    Update all weights going into a single output neuron using gradient descent.

    Args:
        hidden_activations (np.ndarray): Activations from hidden neurons (shape: [n_hidden]).
        weights (np.ndarray): Current weights into the output neuron (shape: [n_hidden]).
        actual_output (float): Current output of this neuron.
        desired_output (float): Target output for this neuron.
        learning_rate (float): Learning rate.

    Returns:
        np.ndarray: Updated weights (shape: [n_hidden]).
    """
    error = actual_output - desired_output
    gradients = error * hidden_activations
    updated_weights = weights - learning_rate * gradients
    updated_weights = np.clip(updated_weights, -0.95, 0.95)
    return updated_weights


def update_single_hidden_neuron_weights(input_activations, weights_to_hidden, downstream_weights, output_errors, hidden_activation, learning_rate=0.01):
    """
    Updates the weights into a single hidden neuron.

    Args:
        input_activations (np.ndarray): Input activations (shape: [n_input]).
        weights_to_hidden (np.ndarray): Weights into this hidden neuron (shape: [n_input]).
        downstream_weights (np.ndarray): Weights from this hidden neuron to each output neuron (shape: [n_output]).
        output_errors (np.ndarray): Errors at each output neuron (shape: [n_output]).
        hidden_activation (float): Activation value of this hidden neuron.
        learning_rate (float): Learning rate.

    Returns:
        np.ndarray: Updated weights for this hidden neuron (shape: [n_input]).
    """
    propagated_error = np.dot(output_errors, downstream_weights)
    derivative = hidden_activation * (1 - hidden_activation)
    local_gradient = propagated_error * derivative
    gradients = local_gradient * input_activations
    updated_weights = weights_to_hidden - learning_rate * gradients
    updated_weights = np.clip(updated_weights, -0.95, 0.95)
    return updated_weights


def update_hidden_layer_weights(spike_times, output_errors):
    lr = 0.01 # learning rate will need to universalize later

    input_activations = np.array(spike_times[:4])
    hidden_activations = np.array(spike_times[4:14])

    for i in range(10):  # 10 hidden neurons
        start_idx = i * 4
        end_idx = (i + 1) * 4

        weights_to_hidden_i = w_2[start_idx:end_idx]
        hidden_activation = hidden_activations[i]

        # FIXED: Correct indexing for downstream weights
        # Each hidden neuron i connects to all 3 outputs with weights at positions:
        # i*3, i*3+1, i*3+2
        downstream_weights = np.array([
            w_3[i * 3 + 0],  # weight from hidden neuron i to output 0
            w_3[i * 3 + 1],  # weight from hidden neuron i to output 1  
            w_3[i * 3 + 2],  # weight from hidden neuron i to output 2
        ])
        
        # Debug: Print the indices being used
        #print(f"  Weight indices for neuron {i}: [{i*3}, {i*3+1}, {i*3+2}]")
        #print(f"  w_3 values at those indices: {w_3[i*3:i*3+3]}")

        propagated_error = np.dot(output_errors, downstream_weights)
        derivative = hidden_activation * (1 - hidden_activation)
        local_gradient = propagated_error * derivative
        gradients = local_gradient * input_activations

        # print(f"\nHidden Neuron {i}")
        # print(f"  Hidden Activation: {hidden_activation}")
        # print(f"  Downstream Weights: {downstream_weights}")
        # print(f"  Propagated Error: {propagated_error}")
        # print(f"  Local Gradient: {local_gradient}")
        # print(f"  Gradients: {gradients}")

        updated_weights = weights_to_hidden_i - lr * gradients
        updated_weights = np.clip(updated_weights, -0.95, 0.95)
        w_2[start_idx:end_idx] = updated_weights

    # print("hidden layer weights updated")


def update_last_layer_weights(spike_times, desired_outputs):

    lr = 0.01 # will make more global later
    
    var = spike_times
    hidden_activations = np.array(var[4:14])


    weights_to_output_0 = w_3[0::3]
    weights_to_output_1 = w_3[1::3]
    weights_to_output_2 = w_3[2::3]

    actual_output_0 = var[-3]
    actual_output_1 = var[-2]
    actual_output_2 = var[-1]


    desired_output_0 = desired_outputs[0]
    desired_output_1 = desired_outputs[1]
    desired_output_2 = desired_outputs[2]


    new_weights_0 = update_weights_input_neuron(
        hidden_activations,
        weights_to_output_0,
        actual_output_0,
        desired_output_0,
        learning_rate=lr
    )

    new_weights_1 = update_weights_input_neuron(
        hidden_activations,
        weights_to_output_1,
        actual_output_1,
        desired_output_1,
        learning_rate=lr
    )

    new_weights_2 = update_weights_input_neuron(
        hidden_activations,
        weights_to_output_2,
        actual_output_2,
        desired_output_2,
        learning_rate=lr
    )

    w_3[0::3] = new_weights_0
    w_3[1::3] = new_weights_1
    w_3[2::3] = new_weights_2

    print("weights updated")

def custom_scale(X, min_val=0.05, max_val=0.95):
    X_min = np.min(X, axis=0)
    X_max = np.max(X, axis=0)
    return min_val + (X - X_min) * (max_val - min_val) / (X_max - X_min)


In [53]:
import numpy as np
import logging

logging.getLogger('brian2').setLevel(logging.ERROR)

# inputs = [0.1, 0.2, 0.5, 0.9]
# wanted_output = [2.1, 2.1, 2.9]

# Initialize weights if not already
# w_1 = np.random.uniform(0.05, 0.95, size=4)
# w_2 = np.random.uniform(-0.95, 0.95, size=40)
# w_3 = np.random.uniform(-0.95, 0.95, size=30)

def real_outputs(var):
    return [round(var[-3 + i] - 2, 4) if var[-3 + i] is not None else 0.0 for i in range(3)]

def get_output_errors(var, wanted_output):
    return np.array([var[-3 + i] - wanted_output[i] for i in range(3)])


# print(w_2)
# for i in range(5):
#     try:
#         var = run_Urd(inputs, w_1, w_2, w_3)

#         print(f"Iter {i} Output:", real_outputs(var))

#         update_last_layer_weights(var, wanted_output)

#         # Compute error for hidden layer backprop
#         output_errors = get_output_errors(var, wanted_output)

#         update_hidden_layer_weights(var, output_errors)

#     except Exception as e:
#         print(f"Error in iteration {i}: {e}")

# # Final output
# print("Final Output:", real_outputs(var))
# print(w_2)


In [54]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

def custom_scale(X, min_val=0.05, max_val=0.95):  # arelayd scaled
    X_min = np.min(X, axis=0)
    X_max = np.max(X, axis=0)
    return min_val + (X - X_min) * (max_val - min_val) / (X_max - X_min)

X_scaled = custom_scale(X)

# Normalize to [0.05, 0.95]
X_min = X.min(axis=0)
X_max = X.max(axis=0)
X_scaled = 0.05 + ((X - X_min) / (X_max - X_min)) * (0.95 - 0.05)

# Combine and shuffle (important!)
combined = list(zip(X_scaled, y))
np.random.seed(42)
np.random.shuffle(combined)

X_shuffled, y_shuffled = zip(*combined)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_shuffled, y_shuffled, test_size=0.2, random_state=42
)

training_data = [list(x) for x in X_train]
result_data = list(y_train)

# Step 3: Define training function (your original code)
def train_Urd(training_data, result_data, loops):
    samples = len(training_data)

    wanted_output = [[-.1, -.1, -.1] for _ in range(samples)]
    
    for i in range(samples):
        if result_data[i] == 0:
            wanted_output[i] = [2.95, 2.05, 2.05]
        elif result_data[i] == 1:
            wanted_output[i] = [2.05, 2.95, 2.05]
        elif result_data[i] == 2:
            wanted_output[i] = [2.05, 2.05, 2.95]
        else:
            print("PRoblemmm")

    for j in range(loops):
        for i in range(samples):
            try:
                var = run_Urd(training_data[i], w_1, w_2, w_3)

                print(f"Iter {i} Output:", real_outputs(var))

                update_last_layer_weights(var, wanted_output[i])

                # calc error for hidden layer backprop
                output_errors = get_output_errors(var, wanted_output[i])

                update_hidden_layer_weights(var, output_errors)

            except Exception as e:
                print(f"Error in iteration {i}: {e}")

        print(f"{j + 1} loop of training data completed")
    
    print(f"{loops} loops completed")


# Step 4: Train your model
train_Urd(training_data, result_data, loops=15)

from collections import defaultdict

correct = 0
total = 0
class_correct = defaultdict(int)
class_total = defaultdict(int)

print("\n--- Evaluation on Test Data ---")
for i, sample in enumerate(X_test):
    pred = run_Urd(sample.tolist(), w_1, w_2, w_3)
    outputs = real_outputs(pred)
    predicted_class = int(np.argmax(outputs))

    actual_class = int(y_test[i]) if isinstance(y_test[i], (int, np.integer)) else int(np.argmax(y_test[i]))

    if predicted_class == actual_class:
        correct += 1
        class_correct[actual_class] += 1
    class_total[actual_class] += 1
    total += 1

    print(f"Sample {i}: Predicted Class: {predicted_class}, Actual Class: {actual_class}, Outputs: {outputs}")

# Final results
print(f"\nOverall Accuracy: {correct}/{total} = {correct / total:.2%}")

for cls in range(3):
    if class_total[cls] > 0:
        acc = class_correct[cls] / class_total[cls]
        print(f"Accuracy for Class {cls}: {class_correct[cls]}/{class_total[cls]} = {acc:.2%}")
    else:
        print(f"Class {cls} has no samples in test set.")

np.savez('weights_checkpoint_15_epocs_Lr=0.01.npz', w1=w_1, w2=w_2, w3=w_3)


Iter 0 Output: [0.354, 0.5, 0.533]
weights updated
Iter 1 Output: [0.488, 0.612, 0.66]
weights updated
Iter 2 Output: [0.345, 0.49, 0.514]
weights updated
Iter 3 Output: [0.319, 0.458, 0.477]
weights updated
Iter 4 Output: [0.492, 0.608, 0.645]
weights updated
Iter 5 Output: [0.424, 0.556, 0.577]
weights updated
Iter 6 Output: [0.562, 0.668, 0.713]
weights updated
Iter 7 Output: [0.539, 0.649, 0.705]
weights updated
Iter 8 Output: [0.462, 0.587, 0.633]
weights updated
Iter 9 Output: [0.461, 0.591, 0.633]
weights updated
Iter 10 Output: [0.45, 0.589, 0.624]
weights updated
Iter 11 Output: [0.312, 0.457, 0.483]
weights updated
Iter 12 Output: [0.48, 0.607, 0.653]
weights updated
Iter 13 Output: [0.498, 0.622, 0.676]
weights updated
Iter 14 Output: [0.482, 0.608, 0.667]
weights updated
Iter 15 Output: [0.31, 0.448, 0.493]
weights updated
Iter 16 Output: [0.512, 0.628, 0.696]
weights updated
Iter 17 Output: [0.459, 0.584, 0.648]
weights updated
Iter 18 Output: [0.411, 0.541, 0.605]
weights

In [55]:
# training_data = [[0.1, 0.5, 0.1, 0.75], [0.3, 0.5, 0.21, 0.65]]
# result_data = [0, 1]

# def train_Urd(training_data, result_data, loops):
#     samples = len(training_data)

#     wanted_output = [[-.1, -.1, -.1] for _ in range(samples)]
    
#     for i in range(samples):
#         if result_data[i] == 0:
#             wanted_output[i] = [0.95, 0.05, 0.05]
#         elif result_data[i] == 1:
#             wanted_output[i] = [0.05, 0.95, 0.05]
#         elif result_data[i] == 2:
#             wanted_output[i] = [0.05, 0.05, 0.95]

#     for j in range(loops):
#         for i in range(samples):
#             try:
#                 var = run_Urd(training_data[i], w_1, w_2, w_3)

#                 print(f"Iter {i} Output:", real_outputs(var))

#                 update_last_layer_weights(var, wanted_output[i])

#                 # calc error for hidden layer backprop
#                 output_errors = get_output_errors(var, wanted_output[i])

#                 update_hidden_layer_weights(var, output_errors)

#             except Exception as e:
#                 print(f"Error in iteration {i}: {e}")

#         print(f"{j + 1} loop of training data completed")
    
#     print(f"{loops} loops completed")



# train_Urd(training_data, result_data, 1)